# Important: Add competition dataset as input dataset first for this notebook to work

In [9]:
!pip install -U bitsandbytes peft accelerate datasets sentencepiece wandb python-dotenv wtpsplit langchain
!pip install flash-attn --no-build-isolation
!pip install wtpsplit==2.1.1
!pip install syntok==1.4.4
!pip install omegaconf
!pip install wandb
!pip install --upgrade transformers trl
!pip install pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 124.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.0
    Uninstalling numpy-2.2.0:
      Successfully uninstalled numpy-2.2.0
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3
  Attempting unin

  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
Using cached transformers-4.46.3-py3-none-any.whl (10.0 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.2
    Uninstalling transformers-4.45.2:
      Successfully uninstalled transformers-4.45.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
adapters 1.0.1 requires transformers~=4.45.2, but you have transformers 4.46.3 which is incompatible.


In [1]:
!git clone https://github.com/Reennon/gen-ai-nlp-lab-1

fatal: destination path 'gen-ai-nlp-lab-1' already exists and is not an empty directory.


In [2]:
%cd gen-ai-nlp-lab-1
!ls

/content/gen-ai-nlp-lab-1
notebooks  params  poetry.lock	pyproject.toml	README.md  src


In [7]:
import os
import torch
import pandas as pd

from langchain_core.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from huggingface_hub import login
from transformers import PreTrainedTokenizerBase, BitsAndBytesConfig
from tqdm import tqdm
from torch.utils.data import Dataset
from datasets import Dataset
from src.prompts.prompts import (NERPrompt1, NERPrompt2, NERPrompt3, NERPrompt4)
from src.prompts.examples import (
    ELEVEN_SHOT_EXAMPLES_DICT,
    FIVE_SHOT_EXAMPLES_DICT,
    THREE_SHOT_EXAMPLES_DICT,
    ZERO_SHOT_EXAMPLES_DICT
)
from omegaconf import OmegaConf
from google.colab import userdata

In [73]:
QUANTIZE_4BIT = False
# device   = "cuda:0"
device = "cuda:0"

In [6]:
parameters = OmegaConf.load("./params/aya_23_8b.yml")

In [8]:
login(userdata.get('hf_key'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [74]:
checkpoint = "CohereForAI/aya-23-8b"
quantization_config = None
if QUANTIZE_4BIT:
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16,
  )
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
seq_length = parameters.baseline.max_new_tokens
tokenizer.model_max_length = seq_length
max_seq_length = seq_length
config = AutoConfig.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    config=config,
    quantization_config=quantization_config,
    torch_dtype="bfloat16",
    device_map="auto",  # Automatically map to GPUs
    attn_implementation="flash_attention_2",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
silver_train_df = pd.read_csv("./data/silver_train.csv")
silver_test_df = pd.read_csv("./data/silver_test.csv")

In [115]:
from src.prompts.prompts import BasePrompt

class NERPrompt1(BasePrompt):
    template: str = """Твоє завдання – виділити всі сутності у наданому тексті за наведеними категоріями та вивести їх у форматі JSON-списку:

Категорії сутностей ("label"):
- ART: артефакт (створений людиною предмет)
- DATE: дата (календарна дата, рік)
- DOC: документ (назви документів)
- JOB: посада (професійний титул, робоча позиція)
- LOG: місце (географічні об’єкти, назви країн, міст, річок тощо)
- MISC: різне (інші сутності, не підпадають під інші категорії)
- MON: гроші (сума, валюта)
- ORG: організація (установи, компанії, заклади)
- PCT: відсоток (число у відсотках)
- PERIOD: період (тривалість часу)
- PERS: особа (людські імена, прізвища)
- QUANT: кількість (числові значення)
- TIME: час (конкретний момент доби)

Формат відповіді: список об’єктів у JSON, кожен об’єкт має поля:
"label" – категорія сутності
"text" – фрагмент тексту сутності з оригінального тексту без змін

Не виводь дублікати знайдених сутностей.
Та не змінюй відмінків знайдених сутностей.
Виводь сутності з вхідного тексту, а не прикладів.
Перевір, чи справді знайдена сутність відповідає категорії сутностей.
Якщо знайдена сутність не відповідає її категорії, то її не слід включати у відповідь.

Нижче наведено приклади формату та стилю розпізнавання сутностей:
{examples}

Вхідний текст:
{text}

Знайдені сутності:"""
    input_variables: list[str] = ["text", "examples"]

In [76]:
example_template: str = """
Вхідний текст:
{example_text}
Знайдені сутності:
{example_labels}
"""

def construct_prompt(
    prompt_template: PromptTemplate,
    few_shot_dict: dict[str, str],
    text: str
) -> str:
    examples = "".join([
            example_template.format(
                example_text=example["text"],
                example_labels=example["labels"]
            ) for example in few_shot_dict
        ])
    prompt = prompt_template.format(
        examples=examples,
        text=text,
    )

    return prompt

prompt = construct_prompt(
    prompt_template=NERPrompt1().prompt_template,
    few_shot_dict=ELEVEN_SHOT_EXAMPLES_DICT,
    text=""
)
print(prompt)

Твоє завдання – виділити всі сутності у наданому тексті за наведеними категоріями та вивести їх у форматі JSON-списку:

Категорії сутностей:
- ART: артефакт (створений людиною предмет)
- DATE: дата (календарна дата, рік)
- DOC: документ (назви документів)
- JOB: посада (професійний титул, робоча позиція)
- LOG: місце (географічні об’єкти, назви країн, міст, річок тощо)
- MISC: різне (інші сутності, не підпадають під інші категорії)
- MON: гроші (сума, валюта)
- ORG: організація (установи, компанії, заклади)
- PCT: відсоток (число у відсотках)
- PERIOD: період (тривалість часу)
- PERS: особа (людські імена, прізвища)
- QUANT: кількість (числові значення)
- TIME: час (конкретний момент доби)

Формат відповіді: список об’єктів у JSON, кожен об’єкт має поля:
"label" – категорія сутності
"text" – фрагмент тексту сутності з оригінального тексту без змін

Не виводь дублікати знайдених сутностей. 
Та не змінюй відмінків знайдених сутностей.
Виводь сутності з вхідного тексту, а не прикладів.

Н

In [77]:
tokenized_prompt = tokenizer.encode(prompt)
decoded_prompt = tokenizer.decode(tokenized_prompt)
prompt_len = len(tokenizer.tokenize(prompt))
print(decoded_prompt), prompt_len

Token indices sequence length is longer than the specified maximum sequence length for this model (1704 > 800). Running this sequence through the model will result in indexing errors


<BOS_TOKEN>Твоє завдання – виділити всі сутності у наданому тексті за наведеними категоріями та вивести їх у форматі JSON-списку:

Категорії сутностей:
- ART: артефакт (створений людиною предмет)
- DATE: дата (календарна дата, рік)
- DOC: документ (назви документів)
- JOB: посада (професійний титул, робоча позиція)
- LOG: місце (географічні об’єкти, назви країн, міст, річок тощо)
- MISC: різне (інші сутності, не підпадають під інші категорії)
- MON: гроші (сума, валюта)
- ORG: організація (установи, компанії, заклади)
- PCT: відсоток (число у відсотках)
- PERIOD: період (тривалість часу)
- PERS: особа (людські імена, прізвища)
- QUANT: кількість (числові значення)
- TIME: час (конкретний момент доби)

Формат відповіді: список об’єктів у JSON, кожен об’єкт має поля:
"label" – категорія сутності
"text" – фрагмент тексту сутності з оригінального тексту без змін

Не виводь дублікати знайдених сутностей. 
Та не змінюй відмінків знайдених сутностей.
Виводь сутності з вхідного тексту, а не пр

(None, 1703)

In [78]:
def get_message_format(prompts):
  messages = []

  for p in prompts:
    messages.append(
        [{"role": "user", "content": p}]
      )

  return messages

In [79]:
import ast
import json
import logging
import re
from typing import Union


def sanitize_json(raw_json: str) -> str:
    """
    Cleans raw JSON by removing characters that could break the structure.
    For example, removes unmatched closing parentheses.
    """
    # Remove extra unmatched closing parentheses
    stack = []
    clean_content = []

    for char in raw_json:
        if char == '(':
            stack.append(char)
        elif char == ')':
            if stack:
                stack.pop()
            else:
                continue  # Skip unmatched closing parentheses
        clean_content.append(char)

    # Remove any dangling commas before a closing brace or bracket
    clean_json = ''.join(clean_content)
    clean_json = re.sub(r',\s*([\}\]])', r'\1', clean_json)

    return clean_json


def try_to_extract_dict_from_json(
    raw_output: str,
) -> Union[list[dict[str, str]], str]:
    pattern = r'```json\s*\n(?P<json>([\s\S]*?))\n```'
    matches = re.finditer(pattern, raw_output, re.MULTILINE)

    dict_output = ""
    for match in matches:
        json_content = match.group('json')
        try:
            dict_output = json.loads(json_content)

            return dict_output
        except json.JSONDecodeError as e:
            logging.debug("Invalid JSON:", e)
        except ValueError as e:
            logging.debug("ValueError:", e)

            return raw_output
    else:
        try:
            dict_output = ast.literal_eval(raw_output.strip("```json"))
        except SyntaxError as e:
            logging.debug(f"Got Syntax Error: {e} for output: {raw_output[:30]}; Sanitizing...")
            raw_output = sanitize_json(raw_output)
            try:
                dict_output = ast.literal_eval(raw_output.strip("```json"))
            except SyntaxError as e:
                logging.debug(f"Got another Syntax Error: {e} for output: {raw_output[:30]}; Returning raw output.")
                return raw_output
        except ValueError as e:
            logging.debug(f"Got Value Error: {e} for output: {raw_output[:30]}; Returning raw output.")

            return raw_output

    return dict_output

In [117]:
out_labels = {}
max_seq_length = prompt_len + seq_length

i = 0

for id in tqdm(silver_test_df.id.unique(), desc="Text progress"):
    i+=1
    if i == 5:
      break
    inputs: list[str] = silver_test_df.loc[silver_test_df.loc[:, "id"] == id, "text"].to_list()
    inputs: list[str] = [construct_prompt(
        prompt_template=NERPrompt1().prompt_template,
        few_shot_dict=THREE_SHOT_EXAMPLES_DICT,
        text=input
    ) for input in inputs]
    print("inputs len")
    print([len(tokenizer.encode(input)) for input in inputs])
    print()
    inputs: list[dict[str, str]] = get_message_format(inputs)
    input_ids = tokenizer.apply_chat_template(
        inputs,
        tokenize=True,
        add_generation_prompt=True,
        padding=True,
        return_tensors="pt",
    )
    input_ids = input_ids.to(model.device)
    #input_ids = input_ids.to(model.device)
    prompt_padded_len = len(input_ids[0])
    # Generate corrections
    # Check if the model is wrapped in DataParallel
    gen_tokens = model.generate(
            input_ids,
            temperature=parameters.baseline.temperature,
            top_p=parameters.baseline.top_p,
            top_k=parameters.baseline.top_k,
            max_new_tokens=seq_length,
            do_sample=True,
    )

    gen_tokens = [
        gt[prompt_padded_len:] for gt in gen_tokens
    ]
    outputs: list[dict] = tokenizer.batch_decode(
        gen_tokens,
        skip_special_tokens=True
    )
    outputs: list[str] = [try_to_extract_dict_from_json(output) for output in outputs]
    print(outputs)
    unique_set = {frozenset(json_item.items()) for json_list in outputs for json_item in outputs}
    # Convert the set back to a list of dictionaries
    outputs = [dict(item) for item in unique_set]

    print("output len")
    print(len(tokenizer.encode(str(outputs))))
    print("output")
    print(id, outputs)

    out_labels[id] = outputs

print("All texts extracted!")

silver_test_predictions_df = pd.DataFrame.from_dict([out_labels]).T
silver_test_predictions_df

Text progress:   0%|          | 0/169 [00:00<?, ?it/s]

inputs len
[1160, 965]



Text progress:   0%|          | 0/169 [00:54<?, ?it/s]

[[{'label': 'ORG', 'text': 'департамент культури і туризму Кіровоградської ОДА'}, {'label': 'LOG', 'text': 'Черкаської області'}, {'label': 'LOG', 'text': 'Звенигородського району'}, {'label': 'LOG', 'text': 'Донбасу'}, {'label': 'DATE', 'text': '2014 р.'}, {'label': 'MISC', 'text': 'Кобзар'}, {'label': 'ORG', 'text': 'літературно-меморіальний музей'}, {'label': 'LOG', 'text': 'Звенигородського району'}, {'label': 'LOG', 'text': 'Донбасу'}, {'label': 'MISC', 'text': 'пана Енгельгарта'}, {'label': 'LOG', 'text': 'Черкаської області'}, {'label': 'LOG', 'text': 'Звенигородського району'}], '[\n    {\n        "label": "ART",\n        "text": "репродукції картин Тараса Григоровича"\n    },\n    {\n        "label": "LOG",\n        "text": "Українське село"\n    },\n    {\n        "label": "LOG",\n        "text": "народний дух"\n    },\n    {\n        "label": "LOG",\n        "text": "Єлисаветград"\n    },\n    {\n        "label": "LOG",\n        "text": "Знам’янка"\n    },\n    {\n        "l

AttributeError: 'list' object has no attribute 'items'

In [119]:
flattened_list = []
for output in outputs:
    if isinstance(output, str):  # Attempt to parse JSON string
        try:
            parsed = json.loads(output)
            flattened_list.extend(parsed)
        except json.JSONDecodeError as e:
            print(f"JSON decoding error: {e}")
            print(f"Problematic string: {output[:500]}")  # Print a snippet for debugging
    elif isinstance(output, list):  # Extend directly if it's already a list
        flattened_list.extend(output)


# Step 2: Remove duplicates based on "label" and "text"
unique_entities = {frozenset(entity.items()): entity for entity in flattened_list}

# Step 3: Convert back to list
result = list(unique_entities.values())
result

JSON decoding error: Unterminated string starting at: line 167 column 9 (char 2586)
Problematic string: [
    {
        "label": "ART",
        "text": "репродукції картин Тараса Григоровича"
    },
    {
        "label": "LOG",
        "text": "Українське село"
    },
    {
        "label": "LOG",
        "text": "народний дух"
    },
    {
        "label": "LOG",
        "text": "Єлисаветград"
    },
    {
        "label": "LOG",
        "text": "Знам’янка"
    },
    {
        "label": "LOG",
        "text": "Кіровоградщина"
    },
    {
        "label": "ORG",
        "text": "музей"
    },
  


[{'label': 'ORG',
  'text': 'департамент культури і туризму Кіровоградської ОДА'},
 {'label': 'LOG', 'text': 'Черкаської області'},
 {'label': 'LOG', 'text': 'Звенигородського району'},
 {'label': 'LOG', 'text': 'Донбасу'},
 {'label': 'DATE', 'text': '2014 р.'},
 {'label': 'MISC', 'text': 'Кобзар'},
 {'label': 'ORG', 'text': 'літературно-меморіальний музей'},
 {'label': 'MISC', 'text': 'пана Енгельгарта'}]

In [114]:
import ast

silver_test_predictions_df = pd.DataFrame.from_dict([out_labels]).T
silver_test_predictions_df.columns = ["entities"]
silver_test_predictions_df.index.name = "id"
silver_test_predictions_df = silver_test_predictions_df.reset_index()

print(ast.literal_eval(str(silver_test_predictions_df.loc[0, "entities"])))

['[\n    {\n        "label": "ORG",\n        "text": "департамент культури і туризму Кіровоградської ОДА"\n    },\n    {\n        "label": "LOG",\n        "text": "Моринці",\n        "text": "Черкаської області"\n    },\n    {\n        "label": "LOG",\n        "text": "Кирилівка",\n        "text": "Звенигородського району"\n    },\n    {\n        "label": "LOG",\n        "text": "Україна",\n        "text": "Криму",\n        "text": "Донбасу"\n    },\n    {\n        "label": "LOG",\n        "text": "Кіровоградщини",\n        "text": "Черкащини"\n    },\n    {\n        "label": "LOG",\n        "text": "Звенигородського району",\n        "text": "Україна"\n    },\n    {\n        "label": "LOG",\n        "text": "Україна",\n        "text": "Криму",\n        "text": "Донбасу"\n    },\n    {\n        "label": "LOG",\n        "text": "Україна",\n        "text": "Кіровоградщини",\n        "text": "Черкащини"\n    },\n    {\n        "label": "LOG",\n        "text": "Україна",\n        "text": "

In [82]:
silver_test_predictions_df.columns = ["entities"]
silver_test_predictions_df.index.name = "id"
silver_test_predictions_df['entities'] = silver_test_predictions_df['entities'].apply(
    lambda nested_list: [item for sublist in nested_list for item in sublist]
)

silver_test_predictions_df.reset_index(inplace=True)
silver_test_predictions_df

ValueError: Length mismatch: Expected axis has 2 elements, new values have 1 elements

In [58]:
silver_test_predictions_df.loc[2]

,2
id,b2c68352957ed341637aa9dd25b8a8ac00c32e8129d689...
entities,"[{'label': 'LOG', 'text': 'Ізмаїл'}, {'label':..."


In [109]:
silver_test_df_concatenated = (
    silver_test_df.groupby("id", as_index=False)["text"].apply(" ".join).reset_index()
)
pd.merge(
    silver_test_predictions_df,
    silver_test_df_concatenated.loc[:, ["text", "id"]],
    how="left",
    on="id"
)

,id,entities,text
0,e29896ab781b5dbb97ae3f3f7862fa681e9d70a5e63866...,"[[{'label': 'MISC', 'text': 'департаменту куль...","Цікаву і пізнавальну подорож місцями , де наро..."
1,d67655fe3fe45e95cd63613c2189fe86728293bd0b8d3c...,"[[{'label': 'MISC', 'text': 'шахова Олімпіада'...",Нещодавно на шаховій Олімпіаді українська жіно...
2,b2c68352957ed341637aa9dd25b8a8ac00c32e8129d689...,"[[{'label': 'LOG', 'text': 'Ізмаїлі'}, {'label...",Виходець із козацько-старшинського роду А . Жд...
3,bd22545c44af9e948961d48f5eacc6ec3c8867f4e21d16...,"[[{'label': 'MISC', 'text': 'кіровоградців'}, ...",На довірливих кіровоградців полюють шахраї та ...


In [64]:
silver_test_predictions_df

,id,entities
0,e29896ab781b5dbb97ae3f3f7862fa681e9d70a5e63866...,"[{'label': 'ORG', 'text': 'Збройні Сили Україн..."
1,d67655fe3fe45e95cd63613c2189fe86728293bd0b8d3c...,"[{'label': 'ORG', 'text': 'Збройні Сили Україн..."
2,b2c68352957ed341637aa9dd25b8a8ac00c32e8129d689...,"[{'label': 'LOG', 'text': 'Ізмаїл'}, {'label':..."
3,bd22545c44af9e948961d48f5eacc6ec3c8867f4e21d16...,"[{'label': 'ORG', 'text': 'Держміграційної слу..."


In [ ]:
import json

def location_baseline(text):
    text = text[0].lower() + text[1:]
    text = text.split(" ")
    text = [item for item in text if item[0].isupper()]
    text = [{"label": "LOC", "text": item} for item in text]
    text = json.dumps(text)
    return text


In [ ]:
df["entities"] = df["text"].apply(location_baseline)
df = df.drop(columns=["text"])
df.to_csv("submission.csv", index=False)